In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import numpy as np
plt.style.use('seaborn-whitegrid')
from utils import PkdbModel, ccolors ,markers, category_filter,convert_unit, \
    caffeine_idx,pktype_data,abs_idx,rel_idx,filter_out,filter_df,group_idx,individual_idx
from scipy import stats


def _regression(t, c):
    """ Linear regression on the log timecourse after maximal value.
    No check is performed if already in equilibrium distribution !.
    The linear regression is calculated from all data points after the maximum.

    :return:
    """
    # TODO: check for distribution and elimination part of curve.
    max_index = np.argmax(c)
    # linear regression
    x = t[max_index :]
    y = np.log(c[max_index :])
    if max_index == (len(c) - 1):
        return [np.nan] * 6
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    return [slope, intercept, r_value, p_value, std_err, max_index]



ImportError: cannot import name 'ccolors'

In [ ]:
from pkdb_app.analysis.pharmacokinetic import f_pk

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
caffeine_tc = PkdbModel("caffeine_timecourse",destination="3-inferred")
caffeine_tc.read()
caffeine_tc.to_array()
caffeine_timecourse = caffeine_tc.data

In [ ]:
caffeine_timecourse.groupby(('oral contraceptives', 'choice')).get_group('Y')[["study","value","mean","unit_intervention","unit","subject_type","time_unit","subject_type"]]

In [ ]:
r_caffeine_idx = (caffeine_timecourse["unit"]=="µg/ml") & (caffeine_timecourse["unit_intervention"]=="mg")& (caffeine_timecourse["time_unit"]=="h")
caffeine_timecourse_abs = caffeine_timecourse[r_caffeine_idx]
caffeine_timecourse_abs["outlier"] = False
caffeine_timecourse_abs.loc[caffeine_timecourse_abs["study"] == "Balogh1992","outlier"] = True

In [ ]:
caffeine_timecourse_abs.groupby("study").get_group("Balogh1992")[["time","value","substance_name","value_intervention","time_unit"]]


In [ ]:
def clearance(kwargs):
    dose = kwargs["value_intervention"]
    t = kwargs["time"]
    if kwargs["subject_type"] == "individual":
        c = kwargs["value"]
    else:
        c = kwargs["mean"]
    [slope, intercept, r_value, p_value, std_err,max_index] =_regression(t,c)
    vd = dose/np.exp(intercept)
    kel = -slope
    clearance = kel*vd
    return clearance
    
    

    

In [ ]:
caffeine_timecourse_abs["clearance"] = caffeine_timecourse_abs.apply(lambda x: clearance(x),axis=1)
caffeine_timecourse_bal = caffeine_timecourse_abs.groupby("study").get_group("Balogh1992") 
caffeine_clr =  PkdbModel("caffeine_clearance",destination="3-inferred")
caffeine_clr.read()
caffeine_clearance = caffeine_clr.data
caffeine_clearance_idx = (caffeine_clearance["unit_intervention"]=="mg") & (caffeine_clearance["unit"]=="l/h")
caffeine_clearance = caffeine_clearance[caffeine_clearance_idx]
caffeine_clearance["clearance"] = caffeine_clearance["value"]
caffeine_clearance_bal = caffeine_clearance.groupby("study").get_group("Balogh1992")
print(len(caffeine_clearance_bal))
print(len(caffeine_timecourse_bal))

caffeine_clearance_concat = pd.concat([caffeine_timecourse_bal,caffeine_clearance_bal], ignore_index=True)

In [ ]:
sns.swarmplot(x="subject_pk", y="clearance", hue="output_type",
              palette=["r", "c"], data=caffeine_clearance_concat)

In [ ]:
caffeine_clearance_bal[["clearance","unit","value_intervention","unit_intervention", "subject_pk"]].head()

In [ ]:
caffeine_timecourse_bal[["clearance","unit","value_intervention","unit_intervention", "subject_pk"]].head()

In [ ]:
caffeine_timecourse_abs.groupby(["unit","unit_intervention","time_unit","subject_type"]).count()


In [ ]:
#ns.lineplot(x="time", y="mean",data=caffeine_timecourse_relative)
# caffeine clearance
fig, ax, = plt.subplots(nrows=1, ncols=1, figsize=(15, 7.5))


df = caffeine_timecourse_abs[ individual_idx(caffeine_timecourse_abs)]
df_group = caffeine_timecourse_abs[ group_idx(caffeine_timecourse_abs)]

for category in ccolors:
    cf_filter = category_filter[category]
    df_category = filter_df(cf_filter,df)
    for idx,row in df_category.iterrows():
        x = row["time"]
        y = row["value"]/row["value_intervention"]

        ax.plot(x, y,color=ccolors[category], linestyle="--", alpha=0.3,label=None)#marker=markers[category],
        #units = df_category["unit"].unique()
        #unit_intervention = df_category["unit_intervention"].unique()


        ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))



    df_category = filter_df(cf_filter,df_group)
    df_category = df_category[df_category["mean"].notnull()]
    for idx,row in df_category.iterrows():
        x = row["time"]
        y = row["mean"]/row["value_intervention"]
        y_err = row["se"]/row["value_intervention"]
        ax.errorbar(x, y,yerr=y_err,color=ccolors[category], alpha=0.6,label=None)#marker=markers[category],
        #units = df_category["unit"].unique()
        #unit_intervention = df_category["unit_intervention"].unique()


    #x_group = df_category["time"]
    #y_group = df_category["mean"]
    #count_group = list(df_category[("weight","count")])



ax.set_title('Concentrations')
ax.set_xlim(left=0)
ax.set_yscale("log")
ax.set_ylim(bottom=0)
#ax.legend()


In [ ]:
caffeine_timecourse_relative["study"].unique()

In [ ]:
caffeine_tc.data.groupby(["subject_type","study"]).apply(lambda x : len(x["pk"].unique()))